In [3]:
import torch
from torch import nn
import time

In [20]:
num_actions = 12
student_hidden_dims = [256, 128, 64]
teacher_hidden_dims = [256, 128, 64]
activation = nn.ELU()
init_noise_std = 0.1
student_cnn_kernel_sizes=[5, 5, 5, 5, 5, 5]
student_cnn_strides=[1, 2, 1, 2, 1, 2]
student_cnn_filters=[32] * 6
student_cnn_paddings=[2, 2, 4, 2, 8, 2]
student_cnn_dilations=[1, 1, 2, 1, 4, 1]
teacher_enc_dims = [128, 64]
len_o1 = 48
len_student_obs = 65
enc_activation = True

s_out_channels = student_cnn_filters
s_in_channels = [len_o1] + student_cnn_filters[:-1]

In [30]:
cnn_student_layers = []
s_cnn_out = len_student_obs - 1
for in_ch, out_ch, kernel_size, stride, padding, dilation in zip(
    s_in_channels, 
    s_out_channels, 
    student_cnn_kernel_sizes, 
    student_cnn_strides, 
    student_cnn_paddings, 
    student_cnn_dilations
):
    cnn_student_layers.append(nn.Conv1d(
        in_channels=in_ch,
        out_channels=out_ch,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        dilation=dilation
    ))
    cnn_student_layers.append(nn.BatchNorm1d(out_ch))
    cnn_student_layers.append(activation)
    print((s_cnn_out + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1, ((s_cnn_out + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1) * out_ch)
    print(((s_cnn_out + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1) * stride >= s_cnn_out) 
    s_cnn_out = (s_cnn_out + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

cnn_student_layers.append(nn.Flatten())
cnn_student_layers.append(nn.Linear(s_cnn_out * s_out_channels[-1], teacher_enc_dims[-1]))
if enc_activation:
    cnn_student_layers.append(activation)
cnn_student = nn.Sequential(*cnn_student_layers)

student_layers = []
student_layers.append(nn.Linear(teacher_enc_dims[-1] + len_o1, student_hidden_dims[0]))
student_layers.append(activation)
for layer_index in range(len(student_hidden_dims)):
    if layer_index == len(student_hidden_dims) - 1:
        student_layers.append(nn.Linear(student_hidden_dims[layer_index], num_actions))
    else:
        student_layers.append(nn.Linear(student_hidden_dims[layer_index], student_hidden_dims[layer_index + 1]))
        student_layers.append(activation)
student = nn.Sequential(*student_layers)

64 2048
True
32 1024
True
32 1024
True
16 512
True
16 512
True
8 256
True


In [27]:
print(sum([p.numel() for p in cnn_student.parameters()]))
print(sum([p.numel() for p in student.parameters()]))
print(sum([p.numel() for p in cnn_student.parameters()]) + sum([p.numel() for p in student.parameters()]))

33856
70860
104716


In [28]:
x = torch.rand(1024, 64, 48)

In [29]:
cnn_student(x.permute(0, 2, 1)).shape

torch.Size([1024, 32, 8])